# Неделя 10 • Natural Language Processing
## Классификация текстов
### Transformers

## Классификация текста с помощью BERT

In [1]:
import gdown
import pandas as pd

file_id = '1O7orH9CrNEhnbnA5KjXji8sgrn6iD5n-'
gdown.download(f'https://drive.google.com/uc?id={file_id}', quiet=False)

# Загрузим CSV-файл
df = pd.read_csv('labeled(1).csv')
df.head()

Downloading...
From: https://drive.google.com/uc?id=1O7orH9CrNEhnbnA5KjXji8sgrn6iD5n-
To: /content/labeled(1).csv
100%|██████████| 4.67M/4.67M [00:00<00:00, 155MB/s]


,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [2]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
import seaborn as sns


# импортируем трансформеры
import transformers
import warnings
warnings.filterwarnings('ignore')

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


### Загружаем модель

Список предобученных моделей: [https://huggingface.co/transformers/pretrained_models.html](https://huggingface.co/transformers/pretrained_models.html)

In [4]:
# pip install transformers sentencepiece
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-tiny-toxicity'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

def text2toxicity(text, aggregate=True):
    """ Calculate toxicity of a text (if aggregate=True) or a vector of toxicity aspects (if aggregate=False)"""
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()
    if isinstance(text, str):
        proba = proba[0]
    if aggregate:
        return 1 - proba.T[0] * (1 - proba.T[-1])
    return proba

print(text2toxicity('я люблю нигеров', True))
# 0.9350118728093193

print(text2toxicity('я люблю нигеров', False))
# [0.9715758  0.0180863  0.0045551  0.00189755 0.9331106 ]

print(text2toxicity(['я люблю нигеров', 'я люблю африканцев'], True))
# [0.93501186 0.04156357]

print(text2toxicity(['я люблю нигеров', 'я люблю африканцев'], False))
# [[9.7157580e-01 1.8086294e-02 4.5550885e-03 1.8975559e-03 9.3311059e-01]
#  [9.9979788e-01 1.9048342e-04 1.5297388e-04 1.7452303e-04 4.1369814e-02]]

0.9350118728093193
[0.9715758  0.01808628 0.0045551  0.00189755 0.9331106 ]
[0.93501186 0.04156357]
[[9.7157592e-01 1.8086283e-02 4.5550955e-03 1.8975539e-03 9.3311059e-01]
 [9.9979788e-01 1.9048377e-04 1.5297372e-04 1.7452303e-04 4.1369792e-02]]


In [6]:
model.eval()

# Список для хранения предсказаний
predictions = []
for text in df['comment']:
  toxicity_score = text2toxicity(text, True)
  predictions.append(toxicity_score)
pred_labels = [1 if score > 0.5 else 0 for score in predictions]

df['toxicity_score'] = predictions
df.head()

,comment,toxic,toxicity_score
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,0.995500
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,0.998225
2,Собаке - собачья смерть\n,1.0,0.986566
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,0.999444
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,0.422284


In [10]:
from sklearn.metrics import classification_report
true_labels = df['toxic'].tolist()
print(classification_report(true_labels, pred_labels, target_names=["Non-toxic", "Toxic"]))

              precision    recall  f1-score   support

   Non-toxic       0.87      0.83      0.85      9586
       Toxic       0.69      0.75      0.72      4826

    accuracy                           0.80     14412
   macro avg       0.78      0.79      0.78     14412
weighted avg       0.81      0.80      0.81     14412

